In [297]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [298]:
def mean_squared_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    if y_true.shape != y_pred.shape:
        raise ValueError("Input shapes do not match.")

    return np.mean((y_true - y_pred) ** 2)

## Dataset extraction

In [299]:
train_file_path = 'train.csv'
test_file_path  = 'test.csv'
test_target_file_path = "sample2.csv"

train_data = np.loadtxt(train_file_path, delimiter=',', skiprows=1)
test_data  = np.loadtxt(test_file_path,  delimiter=',', skiprows=1)

y_train = train_data[:, 1]

X_train = train_data[:, 2:]
X_test  = test_data [:, 1:]

# Display the shape of the target variable and features
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:",  X_test.shape)

Shape of X_train: (9000, 100)
Shape of X_test: (1000, 100)


## Dataset analysis

In [300]:
# for f in range(1, len(X_train[0])):
#     plt.figure(figsize=(4, 3))
#     plt.hist(X_train[:, f], bins=50)

Из графиков видно, что каждый признак имеет одинаковую плотность вероятности, совпадающую с нормальным распределением.

Значит нет смысла использовать разные алгоритмы на различных участках данных, можно использовать один алгоритм "на всём".

Причём так как зависимостей между признаками нет (PCA не работает в данном случае), то можно воспользоваться байесовской линейной регрессией.

$$
y | X, \beta \sim \mathcal{N}(X \beta, \sigma^2 I),
$$

где вектор $\beta$ это вектор, который требуется оценить.

$$
p(\beta | y, X) \propto p(\beta) \cdot p(y | X, \beta) \propto \exp{\left(-\frac{1}{2} (\beta - 0)^T \frac{1}{\tau^2} I (\beta - 0) \right)} 
\cdot \exp{\left(-\frac{1}{2} (y - X \beta)^T \frac{1}{\sigma^2} (y - X \beta) \right)} = 
$$

$$
\exp{\left( -\frac{1}{2\sigma^2}  (y - X \beta)^T (y - X \beta) - \frac{1}{2\tau^2} \|\beta\|^2_2 \right)} \rightarrow max
$$

$$
\beta^* = argmin_{\beta} \left( (y - X \beta)^T (y - X \beta) + \frac{\sigma^2}{\tau^2} \|\beta\|^2_2 \right)
$$

## Training algorithms

In [301]:
alpha_1  = [1e-7, 5 * 1e-7, 1e-6, 4 * 1e-6, 1e-5]
alpha_2  = [1e-7, 5 * 1e-7, 1e-6, 4 * 1e-6, 1e-5]
lambda_1 = [1e-7, 5 * 1e-7, 1e-6, 4 * 1e-6, 1e-5]
lambda_2 = [1e-7, 5 * 1e-7, 1e-6, 4 * 1e-6, 1e-5]

min_mse = 1e20

# for a1 in alpha_1:
#     for a2 in alpha_2:
#         for l1 in lambda_1:
#             for l2 in lambda_2:
#                 model = BayesianRidge(alpha_1=a1, alpha_2=a2, lambda_1=l1, lambda_2=l2)
#                 model.fit(X_train, y_train.ravel())
#                 y_train_pred = model.predict(X_train)
#                 y_test_pred  = model.predict(X_test)

#                 mse_value = mean_squared_error(y_train, y_train_pred)
#                 if mse_value < min_mse:
#                     print("Parameters:", a1, a2, l1, l2)
#                     print("MSE:", mse_value)

#                     min_mse = mse_value

В целом, подбор гиперпараметров вообще ни на что не повлиял.

## Generating output file in valid form

In [302]:
model = BayesianRidge(alpha_1=1e-5, alpha_2=1e-7, lambda_1=1e-7, lambda_2=1e-5)
model.fit(X_train, y_train.ravel())
y_train_pred = model.predict(X_train)
y_test_pred  = model.predict(X_test)

ids = np.array(list(range(len(y_test_pred))))
df_out = pd.DataFrame({'Id': ids, 'Category': y_test_pred})

df_out.to_csv(test_target_file_path, index=False)

print(f"CSV file '{test_target_file_path}' created successfully.")

CSV file 'sample2.csv' created successfully.
